In [1]:
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

## 1. Clone email từ gmail

In [4]:
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

def main():

    creds = None
    messages_data = []

    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
            "credentials.json", SCOPES
        )
            creds = flow.run_local_server(port=0)

    with open("token.json", "w") as token:
        token.write(creds.to_json())    
    try:
        service = build("gmail", "v1", credentials=creds)
        results = (
            service.users().messages().list(userId="me", labelIds=["INBOX"], maxResults=30).execute()
        )
        messages = results.get("messages", [])    
        if not messages:
            print("No messages found.")
            return    
        print("Messages:")
        i = 1
        for message in messages:
            print(f'Message {i} ID: {message["id"]}')
            msg = (
                service.users().messages().get(userId="me", id=message["id"]).execute()
            )
            print(f'  Subject: {msg["payload"]}') 
            i += 1
            messages_data.append([message["id"],msg["payload"]])
    except HttpError as error:
        print(f"An error occurred: {error}")    
    return messages_data
if __name__ == "__main__":
  messages = main()

Messages:
Message 1 ID: 199532165263ea11
  Subject: {'partId': '', 'mimeType': 'multipart/alternative', 'filename': '', 'headers': [{'name': 'Delivered-To', 'value': 'ming2005hn1@gmail.com'}, {'name': 'Received', 'value': 'by 2002:a98:b882:0:b0:243:9be2:e923 with SMTP id z2csp448714eix;        Tue, 16 Sep 2025 08:25:21 -0700 (PDT)'}, {'name': 'X-Received', 'value': 'by 2002:a05:6808:1244:b0:438:386c:5f0a with SMTP id 5614622812f47-43b8da764f4mr7275505b6e.48.1758036321681;        Tue, 16 Sep 2025 08:25:21 -0700 (PDT)'}, {'name': 'ARC-Seal', 'value': 'i=1; a=rsa-sha256; t=1758036321; cv=none;        d=google.com; s=arc-20240605;        b=K2I2II2taApZC77VgMaJ0Yjo/yQzdk9d1E1t2xDDreJSY1GoFiDBMK6f7OAmGLLMcX         Yh14z3RgmzvdLdpgT7vS9XWKoIreFnA3ZS/uTyN9yboEaW1wnUHyndZ8rOXwWkiHLmZv         ZhiVbPAYw/Y076FIys/G/+ykYV/ZXEMl7XlKCK3xse98ECZHlqZbFgLddBmAPbkvERiD         Dk8bH9HNql3rXcZrgipFNh8EHgq8MlcknjSiX/iRiUEQ/FpY2Cl4MNy8gCTB9oFFD/M1         KwpAtVJStJ+bysFEfRWsVFO9zrSGUx7+KKtoi9rb2SUYW09TSB

## 2. Lấy email người gửi

In [7]:
import re
# hàm trích xuất các gmail có trong text
def extract_emails(text):
    emails = re.findall(email_pattern, text)
    return emails

def get_from_header(headers):
    for header in headers:
        if header.get("name") == "From":
            return header.get("value")
    return None  

# tạo định dạng để bắt email trong chuỗi
email_pattern = r'[a-zA-Za-z0-9._%+-]+@[a-zA-Za-z0-9.-]+\.[a-zA-Za-z]{2,}'


headers = messages[0][1]['headers']
from_value = get_from_header(headers)
print(from_value)
email = extract_emails(from_value)
print(f'Email from: {email}')

"Minh Lê Hoàng" <lehoangminh.ai.tech@gmail.com>
Email from: ['lehoangminh.ai.tech@gmail.com']


## 3. Decode nội dung thư

In [5]:
import base64

msg_payload = messages[0][1]['payload'] if 'payload' in messages[0][1] else messages[0][1]
encoded_data = None
if 'parts' in msg_payload and msg_payload['parts']:
    for part in msg_payload['parts']:
        if 'body' in part and 'data' in part['body']:
            encoded_data = part['body']['data']
            print(encoded_data)
            break
if not encoded_data and 'body' in msg_payload and 'data' in msg_payload['body']:
    encoded_data = msg_payload['body']['data']
    print(encoded_data)
if encoded_data:
    decoded_bytes = base64.urlsafe_b64decode(encoded_data)
    decoded_data = decoded_bytes.decode("utf-8", errors="replace")
else:
    decoded_data = ""

print('-'*100)
print(decoded_data)  

S2h1eeG6v24gbcOjaSBz4buRYyBjaMawYSB04burbmcgY8OzOiBtdWEgaMOgbmcga2jDtG5nIGPhuqduIHRy4bqjIHRp4buBbiB0csaw4bubYy4gQuG6oW4gY8Oybg0KxJHGsOG7o2MgdOG6t25nIHRow6ptIG5oaeG7gXUgcXXDoCBo4bqlcCBk4bqrbiBraGkgxJHEg25nIGvDvSBuZ2F5IGjDtG0gbmF5LiBT4buRIGzGsOG7o25nIGPDsyBo4bqhbg0KbsOqbiDGsHUgxJHDo2kgY2jhu4kgZMOgbmggY2hvIGtow6FjaCBow6BuZyBuaGFuaCB0YXkgbmjhuqV0LiBIw6N5IHRydXkgY-G6rXAgd2Vic2l0ZSBj4bunYQ0KY2jDum5nIHTDtGkgxJHhu4Mgbmjhuq1uIHBo4bqnbiB0aMaw4bufbmcgbmdheS4NCg==
----------------------------------------------------------------------------------------------------
Khuyến mãi sốc chưa từng có: mua hàng không cần trả tiền trước. Bạn còn
được tặng thêm nhiều quà hấp dẫn khi đăng ký ngay hôm nay. Số lượng có hạn
nên ưu đãi chỉ dành cho khách hàng nhanh tay nhất. Hãy truy cập website của
chúng tôi để nhận phần thưởng ngay.



## 4. Tiến hành trích xuất 30 email đầu tiên

In [8]:
import pandas as pd
data = {
    'index': [],
    'email_from': [],
    'data': [],
    'label': []
}
for item in range(30):
    data['index'].append(item)
    headers = messages[item][1]['headers']
    from_value = get_from_header(headers)
    email = extract_emails(from_value)
    msg_payload = messages[item][1]['payload'] if 'payload' in messages[item][1] else messages[item][1]
    encoded_data = None
    if 'parts' in msg_payload and msg_payload['parts']:
        for part in msg_payload['parts']:
            if 'body' in part and 'data' in part['body']:
                encoded_data = part['body']['data']
                break
    if not encoded_data and 'body' in msg_payload and 'data' in msg_payload['body']:
        encoded_data = msg_payload['body']['data']
    if encoded_data:
        decoded_bytes = base64.urlsafe_b64decode(encoded_data)
        decoded_data = decoded_bytes.decode("utf-8", errors="replace")
    else:
        decoded_data = ""

    decoded_data = re.sub(r'(\r\n|\n|\r)', ' ', decoded_data)
    data['email_from'].append(email[0])
    data['data'].append(f'{decoded_data}')
    data['label'].append('None')

df = pd.DataFrame(data)

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   index       30 non-null     int64 
 1   email_from  30 non-null     object
 2   data        30 non-null     object
 3   label       30 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.1+ KB
None


In [9]:
labels = ['spam', 'entertainment', 'study', 'friends', 'work', 'advertising']

for i in range(30):
    label = labels[i // 5]
    df.loc[i, 'label'] = label

df

,index,email_from,data,label
0,0,lehoangminh.ai.tech@gmail.com,Khuyến mãi sốc chưa từng có: mua hàng không cầ...,spam
1,1,lehoangminh.ai.tech@gmail.com,Nhận ngay 5 triệu đồng chỉ với vài bước đơn gi...,spam
2,2,lehoangminh.ai.tech@gmail.com,Bạn đã trúng thưởng giải đặc biệt từ chương tr...,spam
3,3,lehoangminh.ai.tech@gmail.com,Thông tin bí mật chỉ dành riêng cho bạn: cơ hộ...,spam
4,4,lehoangminh.ai.tech@gmail.com,Click vào đường link để nhận thưởng ngay lập t...,spam
5,5,lehoangminh.ai.tech@gmail.com,Bộ phim hành động mới vừa ra mắt đã nhận được ...,entertainment
6,6,lehoangminh.ai.tech@gmail.com,Sự kiện giải trí lớn nhất năm sẽ diễn ra vào t...,entertainment
7,7,lehoangminh.ai.tech@gmail.com,Danh sách nhạc hot tuần này đã được cập nhật t...,entertainment
8,8,lehoangminh.ai.tech@gmail.com,Tuần này có nhiều bộ phim mới ra mắt với nội d...,entertainment
9,9,lehoangminh.ai.tech@gmail.com,Tham gia sự kiện cuối tuần này để nhận nhiều p...,entertainment


## 6. Lưu lại data đã trích xuất được

In [10]:
df.to_csv('30_emails.csv', index=False)